In [1]:
import requests
from bs4 import BeautifulSoup
import re
import rasterio
import pandas as pd

In [2]:
url = "https://www.geopunt.be/download?container=dhm-vlaanderen-ii-dsm-raster-1m&title=Digitaal%20Hoogtemodel%20Vlaanderen%20II,%20DSM,%20raster,%201m"

url

'https://www.geopunt.be/download?container=dhm-vlaanderen-ii-dsm-raster-1m&title=Digitaal%20Hoogtemodel%20Vlaanderen%20II,%20DSM,%20raster,%201m'

In [3]:
r = requests.get(url)
# I display the requested url and the return of the server
print(url, r.status_code)
# I ask beautifulsoup to store the HTML content of the website in the `soup` variable
soup = BeautifulSoup(r.content, "html")
soup

https://www.geopunt.be/download?container=dhm-vlaanderen-ii-dsm-raster-1m&title=Digitaal%20Hoogtemodel%20Vlaanderen%20II,%20DSM,%20raster,%201m 200



<!DOCTYPE html>

<!--[if lt IE 7]>      <html class="no-js lt-ie9 lt-ie8 lt-ie7" lang="nl"> <![endif]-->
<!--[if IE 7]>         <html class="no-js lt-ie9 lt-ie8" lang="nl"> <![endif]-->
<!--[if IE 8]>         <html class="no-js lt-ie9" lang="nl"> <![endif]-->
<!--[if gt IE 8]><!-->
<html class="no-js" lang="nl">
<!--<![endif]-->
<head>
<meta charset="utf-8"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=1, user-scalable=no" name="viewport"/>
<link href="/img/Geo/favicon.ico" rel="icon" type="image/ico"/>
<link href="/css/Geo/normalize.min.css?v=2.3.0.29529" rel="stylesheet"/>
<!-- 1140px Grid styles for IE -->
<!--[if lte IE 9]><link rel="stylesheet" href="/css/Geo/ie.css?v=2.3.0.29529" type="text/css" media="screen" /><![endif]-->
<link href="/css/Geo/1140.css?v=2.3.0.29529" rel="stylesheet"/>
<link href="/css/Geo/jquery-ui-1.9.2.custom.css" rel="stylesheet">
<link href="/css/Geo/main.css?v=2.3.0.2

In [4]:
zip_links = []
for elem in soup.find_all("li"):
    zip_links.append(elem.find_all("a")[0].get("href"))
    
    
    
zip_links


# for elem in soup.select("li.search-results__item a.card__title-link"):
        #href_listing = elem.get("href")
        #index_search = href_listing.find("?searchId")

['https://downloadagiv.blob.core.windows.net/dhm-vlaanderen-ii-dsm-raster-1m/DHMVIIDSMRAS1m_k01.zip',
 'https://downloadagiv.blob.core.windows.net/dhm-vlaanderen-ii-dsm-raster-1m/DHMVIIDSMRAS1m_k02.zip',
 'https://downloadagiv.blob.core.windows.net/dhm-vlaanderen-ii-dsm-raster-1m/DHMVIIDSMRAS1m_k03.zip',
 'https://downloadagiv.blob.core.windows.net/dhm-vlaanderen-ii-dsm-raster-1m/DHMVIIDSMRAS1m_k04.zip',
 'https://downloadagiv.blob.core.windows.net/dhm-vlaanderen-ii-dsm-raster-1m/DHMVIIDSMRAS1m_k05.zip',
 'https://downloadagiv.blob.core.windows.net/dhm-vlaanderen-ii-dsm-raster-1m/DHMVIIDSMRAS1m_k06.zip',
 'https://downloadagiv.blob.core.windows.net/dhm-vlaanderen-ii-dsm-raster-1m/DHMVIIDSMRAS1m_k07.zip',
 'https://downloadagiv.blob.core.windows.net/dhm-vlaanderen-ii-dsm-raster-1m/DHMVIIDSMRAS1m_k08.zip',
 'https://downloadagiv.blob.core.windows.net/dhm-vlaanderen-ii-dsm-raster-1m/DHMVIIDSMRAS1m_k09.zip',
 'https://downloadagiv.blob.core.windows.net/dhm-vlaanderen-ii-dsm-raster-1m/DHMVI

In [ ]:
bounds = []
for link in zip_links[0:1]:
    print(link)
    
    m = re.search("/dhm-vlaanderen-ii-dsm-raster-1m/(?P<zip_file_name>[\w\-]+).zip$", link)
    zip_file_name = m.group("zip_file_name")
    print(zip_file_name)
    tif_request = "zip+" + link + "!/GeoTIFF/" + zip_file_name + ".tif"
    print(tif_request)
    
    img_DTM = rasterio.open(tif_request)
    print(img_DTM.bound)
    
    bounds.append(img_DTM.bound)
    #/dhm-vlaanderen-ii-dsm-raster-1m/DHMVIIDSMRAS1m_k01.zip
#img_DTM = rasterio.open('zip+https://downloadagiv.blob.core.windows.net/dhm-vlaanderen-ii-dsm-raster-1m/DHMVIIDSMRAS1m_k01.zip!/GeoTIFF/DHMVIIDSMRAS1m_k01.tif')
#img_DTM.bounds

https://downloadagiv.blob.core.windows.net/dhm-vlaanderen-ii-dsm-raster-1m/DHMVIIDSMRAS1m_k01.zip
DHMVIIDSMRAS1m_k01
zip+https://downloadagiv.blob.core.windows.net/dhm-vlaanderen-ii-dsm-raster-1m/DHMVIIDSMRAS1m_k01.zip!/GeoTIFF/DHMVIIDSMRAS1m_k01.tif


In [ ]:
bounds

In [ ]:
f'{1:05d}'

In [ ]:
# Check if DTM files have the same boundaries
# Load them in a dataframe

In [ ]:
df = pd.DataFrame({"URL": zip_links})
df["Boundaries"] = bounds

In [ ]:
# Save metadata in a dataframe

In [ ]:
# Save the dataframe to a csv file
df.to_csv("./assets/meta_data_blocks.csv", index=False)